In [5]:
# ============================================================
# xgb1.py – Training + Prediksi 2024 + Evaluasi Hold-Out 2023
# ============================================================
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import GroupKFold
import xgboost as xgb
import joblib, pathlib, matplotlib.pyplot as plt, warnings
warnings.filterwarnings("ignore")

# ---------- 1. LOAD DATA ----------
FILE = r"prediksi permintaan (3).xlsx"
df = pd.read_excel(FILE, sheet_name="Sheet1")
df.columns = df.columns.str.strip()

konsumsi_col = [c for c in df.columns if "konsumsi" in c.lower() and "ton" in c.lower()][0]
df["konsumsi (ton)"] = pd.to_numeric(df[konsumsi_col], errors="coerce")

FEAT = ["curah hujan (mm)", "suhu (C)", "produksi", "Jumlah Penduduk", "PDRB (tahunan)"]
COLS = ["Provinsi", "Tahun", "Komoditas"] + FEAT + ["konsumsi (ton)"]
df = df[COLS].dropna()
df["Tahun"] = df["Tahun"].astype(int)

# ---------- 2. LAG FEATURE ----------
df = df.sort_values(["Provinsi", "Komoditas", "Tahun"])
df["lag1"] = df.groupby(["Provinsi", "Komoditas"])["konsumsi (ton)"].shift(1)
df["lag2"] = df.groupby(["Provinsi", "Komoditas"])["konsumsi (ton)"].shift(2)
df = df.dropna()

# ---------- 3. FUNGSI MAPE ----------
def hitung_mape(y_true, y_pred):
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# ---------- 4. PISAH HOLD-OUT (2023) & TRAINING (≤ 2022) ----------
train_df = df[df["Tahun"] <= 2022].copy()
test_df  = df[df["Tahun"] == 2023].copy()

# ---------- 5. TRAINING + CV ----------
hasil_eval = []
MODEL_DIR = pathlib.Path("model_output")
MODEL_DIR.mkdir(exist_ok=True)
feat_final = FEAT + ["lag1", "lag2"]

for kom in train_df["Komoditas"].unique():
    sub_train = train_df[train_df["Komoditas"] == kom].copy()
    X = sub_train[feat_final]
    y = sub_train["konsumsi (ton)"]
    y_log = np.log1p(y)

    groups = sub_train["Provinsi"].factorize()[0]
    gkf = GroupKFold(n_splits=5)
    oof = np.zeros(len(sub_train))

    for fold, (tr_idx, vl_idx) in enumerate(gkf.split(X, y_log, groups)):
        X_tr, X_vl = X.iloc[tr_idx], X.iloc[vl_idx]
        y_tr, y_vl = y_log.iloc[tr_idx], y_log.iloc[vl_idx]

        xgb_reg = xgb.XGBRegressor(
            n_estimators=200, learning_rate=0.05, max_depth=3,
            subsample=0.6, colsample_bytree=0.6, reg_lambda=5.0,
            objective="reg:tweedie", tweedie_variance_power=1.2,
            eval_metric="mae", random_state=42, n_jobs=-1,
        )
        xgb_reg.fit(X_tr, y_tr)
        oof[vl_idx] = xgb_reg.predict(X_vl)

    sub_train["pred_ton"] = np.expm1(oof)
    mae = mean_absolute_error(y, sub_train["pred_ton"])
    mape = hitung_mape(y, sub_train["pred_ton"])
    r2 = r2_score(y, sub_train["pred_ton"])
    mse = mean_squared_error(y, sub_train["pred_ton"])
    hasil_eval.append({"Komoditas": kom, "R2": r2, "MAE": mae, "MAPE": mape, "MSE": mse})

    # model penuh
    full_model = xgb.XGBRegressor(**xgb_reg.get_params())
    full_model.fit(X, y_log)
    joblib.dump({"model": full_model, "feat": feat_final}, MODEL_DIR / f"konsumsi_{kom}.pkl")

# ---------- 6. EVALUASI HOLD-OUT 2023 ----------
hold_out = []
for kom in test_df["Komoditas"].unique():
    sub_test = test_df[test_df["Komoditas"] == kom].copy()
    if len(sub_test) == 0:
        continue
    mdl = joblib.load(MODEL_DIR / f"konsumsi_{kom}.pkl")
    model, feat = mdl["model"], mdl["feat"]

    X_test = sub_test[feat]
    y_true = sub_test["konsumsi (ton)"]
    y_pred = np.expm1(model.predict(X_test))

    mae = mean_absolute_error(y_true, y_pred)
    mape = hitung_mape(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    hold_out.append({"Komoditas": kom, "R2": r2, "MAE": mae, "MAPE": mape, "MSE": mse})

hold_df = pd.DataFrame(hold_out).round(2)
all_y_true, all_y_pred = [], []
for kom in test_df["Komoditas"].unique():
    sub_test = test_df[test_df["Komoditas"] == kom].copy()
    if len(sub_test) == 0:
        continue
    mdl = joblib.load(MODEL_DIR / f"konsumsi_{kom}.pkl")
    model, feat = mdl["model"], mdl["feat"]
    X_test = sub_test[feat]
    y_true = sub_test["konsumsi (ton)"]
    y_pred = np.expm1(model.predict(X_test))
    all_y_true.extend(y_true)
    all_y_pred.extend(y_pred)

total_mae = mean_absolute_error(all_y_true, all_y_pred)
total_mse = mean_squared_error(all_y_true, all_y_pred)
total_rmse = np.sqrt(total_mse)
total_mape = hitung_mape(np.array(all_y_true), np.array(all_y_pred))
total_r2 = r2_score(all_y_true, all_y_pred)

print("\n=== Evaluasi Hold-Out 2023 per Komoditas ===")
print(hold_df)
print("\n=== TOTAL METRIK HOLD-OUT 2023 ===")
print(f"TOTAL MAE : {total_mae:,.2f} ton")
print(f"TOTAL MSE : {total_mse:,.2f}")
print(f"TOTAL RMSE: {total_rmse:,.2f} ton")
print(f"TOTAL MAPE: {total_mape:.2f}%")
print(f"TOTAL R²  : {total_r2:.2f}")

# ---------- 7. PREDIKSI 2024 ----------
df_2023 = df[df["Tahun"] == 2023].copy()
df_2024 = df_2023.copy()
df_2024["Tahun"] = 2024
df_2024["lag2"] = df_2023["lag1"]
df_2024["lag1"] = df_2023["konsumsi (ton)"]

pred_2024 = []
for kom in df["Komoditas"].unique():
    mdl = joblib.load(MODEL_DIR / f"konsumsi_{kom}.pkl")
    model, feat = mdl["model"], mdl["feat"]

    sub = df_2024[df_2024["Komoditas"] == kom].copy()
    X_new = sub[feat]
    sub["Konsumsi_2024"] = np.expm1(model.predict(X_new))   # <-- PENTING: agar forecast 5 th tidak error
    pred_2024.append(sub[["Provinsi", "Komoditas", "Konsumsi_2024"]])

df_pred = pd.concat(pred_2024, ignore_index=True)
df_2024 = df_2024.merge(df_pred, on=["Provinsi", "Komoditas"])

df_2024["Produksi_2024"] = df_2024["produksi"]
df_2024["Surplus_2024"] = df_2024["Produksi_2024"] - df_2024["Konsumsi_2024"]
df_2024["Status"] = df_2024["Surplus_2024"].apply(lambda x: "Surplus" if x > 0 else "Defisit")

# ---------- 8. RINGKASAN PER PROVINSI ----------
ringkas = (
    df_2024.groupby("Provinsi", as_index=False)
    .agg(Produksi_2024=("Produksi_2024", "sum"),
         Konsumsi_2024=("Konsumsi_2024", "sum"),
         Surplus_2024=("Surplus_2024", "sum"))
)
ringkas["Status"] = ringkas["Surplus_2024"].apply(lambda x: "Surplus" if x > 0 else "Defisit")

# ---------- 9. TOP-10 ----------
print("\n=== Top 10 Surplus 2024 (per komoditas) ===")
print(df_2024.nlargest(10, "Surplus_2024")[["Provinsi", "Komoditas", "Surplus_2024"]])

print("\n=== Top 10 Defisit 2024 (per komoditas) ===")
print(df_2024.nsmallest(10, "Surplus_2024")[["Provinsi", "Komoditas", "Surplus_2024"]])

print("\n=== Top 10 Surplus 2024 (total provinsi) ===")
print(ringkas.nlargest(10, "Surplus_2024")[["Provinsi", "Surplus_2024"]])

print("\n=== Top 10 Defisit 2024 (total provinsi) ===")
print(ringkas.nsmallest(10, "Surplus_2024")[["Provinsi", "Surplus_2024"]])

# ---------- 10. EXPORT ----------
REPORT_DIR = pathlib.Path("report")
REPORT_DIR.mkdir(exist_ok=True)
hold_df.to_excel(REPORT_DIR / "Evaluasi_HoldOut_2023.xlsx", index=False)
df_2024.to_excel(REPORT_DIR / "Surplus_Defisit_2024_Detail.xlsx", index=False)
ringkas.to_excel(REPORT_DIR / "Surplus_Defisit_2024_Ringkasan_Provinsi.xlsx", index=False)
print("\n✅ File tersimpan:")
print("- Evaluasi_HoldOut_2023.xlsx")
print("- Surplus_Defisit_2024_Detail.xlsx")
print("- Surplus_Defisit_2024_Ringkasan_Provinsi.xlsx")


=== Evaluasi Hold-Out 2023 per Komoditas ===
   Komoditas    R2        MAE   MAPE           MSE
0      Beras  0.91  109806.55  31.84  7.102465e+10
1     Jagung  0.74    2370.54  68.81  2.328551e+07
2    Kedelai  0.82   15958.16  51.06  1.711878e+09
3    Kentang  0.94    4170.29  52.21  5.776148e+07
4   Singkong  0.75   18046.61  27.62  2.619074e+09
5  Ubi Jalar  0.62   13028.13  43.06  1.890000e+09

=== TOTAL METRIK HOLD-OUT 2023 ===
TOTAL MAE : 27,230.05 ton
TOTAL MSE : 12,887,774,401.17
TOTAL RMSE: 113,524.33 ton
TOTAL MAPE: 45.77%
TOTAL R²  : 0.93

=== Top 10 Surplus 2024 (per komoditas) ===
             Provinsi Komoditas  Surplus_2024
112           Lampung  Singkong  7.225544e+06
66         Jawa Timur     Beras  5.298142e+06
60        Jawa Tengah     Beras  5.099350e+06
67         Jawa Timur    Jagung  4.785684e+06
64        Jawa Tengah  Singkong  2.650572e+06
162  Sulawesi Selatan     Beras  2.508923e+06
61        Jawa Tengah    Jagung  2.173594e+06
108           Lampung     Ber

In [7]:
# ============================================================
# forecast_5th.py – ROLLING FORECAST 5 TAHUN (2024-2028)
# ============================================================
import pandas as pd
import numpy as np
import joblib, pathlib, matplotlib.pyplot as plt
from tqdm import tqdm   # progress bar (boleh dihapus)

import pathlib, os, sys
BASE_DIR    = pathlib.Path.cwd()
DETAIL_FILE = BASE_DIR / "report" / "Surplus_Defisit_2024_Detail.xlsx"

if not DETAIL_FILE.exists():
    sys.exit(f"❌ File tidak ditemukan: {DETAIL_FILE}\nPastikan jalankan xgb1.py dulu!")

df_roll = pd.read_excel(DETAIL_FILE)

# ---------- 2. ambil feature dari training ----------
TRAIN_FILE = r"prediksi permintaan (3).xlsx"
df_train = pd.read_excel(TRAIN_FILE)

FEAT = ["curah hujan (mm)", "suhu (C)", "produksi", "Jumlah Penduduk", "PDRB (tahunan)"]
feat_avg = (df_train[df_train["Tahun"] == 2023]
            .groupby(["Provinsi", "Komoditas"])[FEAT].mean().reset_index())

# merge hanya feature
df_roll = df_roll.merge(feat_avg, on=["Provinsi", "Komoditas"], how="left", suffixes=("", "_train"))

# isi kolom yang memang tidak ada / NaN
for col in FEAT:
    if col not in df_roll.columns:
        df_roll[col] = df_roll[col + "_train"]
    else:
        df_roll[col] = df_roll[col].fillna(df_roll[col + "_train"])

# bersihkan kolom bantu
df_roll = df_roll.drop(columns=[c for c in df_roll.columns if c.endswith("_train")])

# ---------- 3. fungsi bantu ----------
def prediksi_tahun(df_in, tahun):
    """
    df_in : dataframe 1 th, kolom lengkap termasuk lag1 & lag2
    return: df_in + kolom Konsumsi_{tahun}, Surplus_{tahun}, Status
    """
    kom_list = df_in["Komoditas"].unique()
    out = []
    for kom in tqdm(kom_list, desc=f"Forecast {tahun}"):
        sub = df_in[df_in["Komoditas"] == kom].copy()
        mdl = joblib.load(MODEL_DIR / f"konsumsi_{kom}.pkl")
        model, feat = mdl["model"], mdl["feat"]

        X = sub[feat]
        pred_log = model.predict(X)
        pred_ton = np.expm1(pred_log)

        sub[f"Konsumsi_{tahun}"] = pred_ton
        sub[f"Produksi_{tahun}"] = sub["produksi"]      # asumsi tetap = 2023
        sub[f"Surplus_{tahun}"]  = sub[f"Produksi_{tahun}"] - sub[f"Konsumsi_{tahun}"]
        sub[f"Status_{tahun}"]   = sub[f"Surplus_{tahun}"].apply(lambda x: "Surplus" if x > 0 else "Defisit")
        out.append(sub)
    return pd.concat(out, ignore_index=True)

# ---------- 4. rolling 5 th ----------
years = [2024, 2025, 2026, 2027, 2028]
MODEL_DIR = pathlib.Path("model_output")

# ---------- pastikan lag1 & lag2 ada ----------
if "lag1" not in df_roll.columns:
    df_roll["lag1"] = df_roll["Konsumsi_2024"]   # base 2024 sudah ada
if "lag2" not in df_roll.columns:
    df_roll["lag2"] = df_roll["lag2"].fillna(0)  # atau shift sendiri

for y in years:
    if y > 2024:
        # update lag
        df_roll["lag2"] = df_roll["lag1"]
        df_roll["lag1"] = df_roll[f"Konsumsi_{y-1}"]
        # (opsional) update feature iklim/ekonomi jika punya proyeksi
        # df_roll[FEAT[:-2]] = proyeksi_df[FEAT[:-2]].iloc[0]
    df_roll = prediksi_tahun(df_roll, y)

# ---------- 5. agregat nasional (contoh komoditas) ----------
KOM = "Beras"
cols_kons = [f"Konsumsi_{y}" for y in years]
cols_stok = [f"Surplus_{y}" for y in years]
nasional = (df_roll[df_roll["Komoditas"] == KOM]
            .groupby("Provinsi")[cols_kons + cols_stok].sum()
            .sum(axis=0))          # sum per tahun

plot_df = pd.DataFrame({
    "Tahun"     : years,
    "Kebutuhan" : nasional[cols_kons].values,
    "Stok"      : nasional[cols_stok].values
})

# ---------- 6. GRAFIK ----------
plt.style.use("seaborn-v0_8")
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(plot_df["Tahun"], plot_df["Kebutuhan"], marker="o", label="Kebutuhan", color="crimson")
ax.plot(plot_df["Tahun"], plot_df["Stok"], marker="s", label="Stok", color="seagreen")
ax.axvspan(2023.5, 2028.5, alpha=0.2, color="gray", label="Forecast 5 th")

ax.set_title(f"Rolling Forecast Kebutuhan vs Stok {KOM} (2024-2028)", fontsize=14)
ax.set_xlabel("Tahun")
ax.set_ylabel("Ton")
ax.legend()
ax.grid(alpha=.3)

# ---------- 7. SIMPAN ----------
REPORT_DIR = pathlib.Path("report")
REPORT_DIR.mkdir(exist_ok=True)
out_png = REPORT_DIR / f"time_series_5th_{KOM.lower()}.png"
fig.savefig(out_png, dpi=300, bbox_inches="tight")
plt.close(fig)

out_excel = REPORT_DIR / f"forecast_5th_{KOM.lower()}.xlsx"
df_roll.to_excel(out_excel, index=False)

print("✅ Forecast 5 th selesai:")
print(" - Grafik :", out_png.resolve())
print(" - Excel  :", out_excel.resolve())

Forecast 2028: 100%|██████████| 6/6 [00:00<00:00, 89.30it/s]


✅ Forecast 5 th selesai:
 - Grafik : D:\SEC\report\time_series_5th_beras.png
 - Excel  : D:\SEC\report\forecast_5th_beras.xlsx
